In [0]:
!pip install gym

In [0]:
!pip install pyvirtualdisplay
!pip install piglet

In [0]:
!apt-get install -y python-numpy python-dev cmake zlib1g-dev libjpeg-dev xvfb ffmpeg xorg-dev python-opengl libboost-all-dev libsdl2-dev swig

In [12]:
!apt-get install xdpyinfo

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package xdpyinfo


In [11]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [14]:
# -*- coding: utf-8 -*-
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

EPISODES = 1000

Using TensorFlow backend.


In [0]:
# Deep Q-learning Agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()
    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
              target = reward + self.gamma * \
                       np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [0]:
if __name__ == "__main__":
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    # agent.load("./save/cartpole-dqn.h5")
    done = False
    batch_size = 32

    for e in range(EPISODES):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for time in range(500):
            # env.render()
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -10
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done:
                print("episode: {}/{}, score: {}, e: {:.2}"
                      .format(e, EPISODES, time, agent.epsilon))
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)
        # if e % 10 == 0:
        #     agent.save("./save/cartpole-dqn.h5")





episode: 0/1000, score: 32, e: 1.0








episode: 1/1000, score: 30, e: 0.86
episode: 2/1000, score: 24, e: 0.76
episode: 3/1000, score: 10, e: 0.73
episode: 4/1000, score: 7, e: 0.7
episode: 5/1000, score: 32, e: 0.6
episode: 6/1000, score: 10, e: 0.57
episode: 7/1000, score: 9, e: 0.54
episode: 8/1000, score: 13, e: 0.51
episode: 9/1000, score: 19, e: 0.46
episode: 10/1000, score: 23, e: 0.41
episode: 11/1000, score: 31, e: 0.35
episode: 12/1000, score: 10, e: 0.34
episode: 13/1000, score: 12, e: 0.32
episode: 14/1000, score: 100, e: 0.19
episode: 15/1000, score: 42, e: 0.15
episode: 16/1000, score: 48, e: 0.12
episode: 17/1000, score: 47, e: 0.096
episode: 18/1000, score: 89, e: 0.062
episode: 19/1000, score: 100, e: 0.037
episode: 20/1000, score: 49, e: 0.029
episode: 21/1000, score: 96, e: 0.018
episode: 22/1000, score: 166, e: 0.01
episode: 23/1000, score: 157, e: 0.01
episode: 24/1000, score: 499, e: 0.01
episode: 25/1000, score: 247, e: 0.01
episode: 26/1000, score: 314, 

In [0]:
agent.save("./save/cartpole-dqn.h5")